In [2]:
!pip3 install cvzone
!pip3 install mediaPipe


  Preparing metadata (setup.py) ... done
  Created wheel for cvzone: filename=cvzone-1.6.1-py3-none-any.whl size=26298 sha256=58bda37a32b0f281e29d19985702868eae68fd5595b4c2ba960430cff29d8624
  Stored in directory: /root/.cache/pip/wheels/2c/9f/b3/92e945ac4a71bf727a92463f38155cc5a4fa49c5010b38ec4c
Successfully built cvzone
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 31.4 MB/s eta 0:00:00


In [3]:
!pip install protobuf==3.20.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.62.3 requires protobuf>=4.21.6, but you have protobuf 3.20.3 which is incompatible.
mediapipe 0.10.20 requires protobuf<5,>=4.25.3, but you have protobuf 3.20.3 which is incompatible.


In [4]:
import cv2
import numpy as np
from math import sqrt
from cvzone.PoseModule import PoseDetector
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow


In [5]:
def distance(a, b):
    return(sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2 + (a[2] - b[2]) ** 2))

def angle(a, b, c):
    ca = a - c
    bc = b - c
    cosine_angle = np.dot(ca, bc) / (np.linalg.norm(ca) * np.linalg.norm(bc))
    return cosine_angle


In [6]:

# li2 = []
# for i in range(50):
#   success, img = cap.read()

#   if not success:
#     break

#   img = detector.findPose(img)
#   lmList, bboxInfo = detector.findPosition(img)

#   if bboxInfo:
#     lm_left_ankle = lmList[28]
#     lm_right_ankle = lmList[27]
#     # finding distance between them
#     point_2 = distance(lm_right_ankle, lm_left_ankle)

#     # exponential filter
#     point_1 = 0.5 * point_1 + 0.5 * point_2
#     li2.append(point_1)
#     cv2_imshow(img)

# if li2:
#   maxi = max(li2)
#   mini = min(li2)
#   threshold = (maxi + mini) / 2
# else:
#   print("Error: Video not found.")

# steps = 2
# time = 50
# peak_loc = 0


In [7]:

# while True:
#   success, img = cap.read()

#   if not success:
#     print("Video ended.")
#     break

#   img = detector.findPose(img)
#   lmList, bboxInfo = detector.findPosition(img)

#   time += 1

#   if bboxInfo:
#     #print(lmList)
#     lm_right_hip = lmList[23]
#     lm_left_hip = lmList[24]
#     lm_right_knee = lmList[25]
#     lm_left_knee = lmList[26]
#     lm_left_ankle = lmList[28]
#     lm_right_ankle = lmList[27]

# #finding angle between them
#     # point_2 = angle(np.array(lm_right_knee), np.array(lm_left_knee), (np.array(lm_right_hip) + np.array(lm_left_hip)) / 2)

#     #finding distance between them
#     point_2 = distance(lm_right_ankle, lm_left_ankle)

#     #exponential filter
#     point_1_exp = 0.5 * point_1 + 0.5 * point_2
#     li.append(point_1_exp)

#     if point_1_exp > threshold and point_1 < threshold:
#         if time - peak_loc >= 12:      #GOTTA MAKE THIS 12 DYNAMIC
#             steps += 1
#             peak_loc = time
#             threshold = 0.25 * (maxi+mini) + 0.5 * threshold
#             print(threshold)
#             maxi = point_1
#             mini = point_1

#     if point_1_exp > maxi:
#         maxi = point_1_exp
#     if point_1_exp < mini:
#         mini = point_1_exp


#     point_1 = point_1_exp

#     #cv2_imshow(img)
#     key = cv2.waitKey(1)
#     if key == 27:
#         break


In [11]:

def initialize_threshold(cap, detector, num_frames=60):
    li2 = []
    point_1 = 0  # Initial point_1 value
    j = num_frames

    for _ in range(num_frames):
        success, img = cap.read()
        if not success:
            break

        img = detector.findPose(img)
        lmList, bboxInfo = detector.findPosition(img)

        if bboxInfo:
            lm_left_ankle = lmList[28]
            lm_right_ankle = lmList[27]

            # Calculate distance between ankles
            point_2 = distance(lm_right_ankle, lm_left_ankle)

            # Apply exponential filter
            point_1 = 0.5 * point_1 + 0.5 * point_2
            li2.append(point_1)
            #cv2_imshow(img)
        elif not bboxInfo:
            j = j-1;
            print("Human not detected.")
            continue;

    if li2:
        maxi = max(li2)
        mini = min(li2)
        threshold = (maxi + mini) / 2
        return threshold, li2[num_frames - 1]
    else:
        raise ValueError("Error: Video not found or unable to read frames.")


In [12]:
def main_processing_loop(cap, detector, threshold, steps=2, time=60, peak_loc=0, point_1 = 0):
    maxi = 0
    mini = 0
    #li = []
    while True:
        success, img = cap.read()
        if not success:
            print("Video ended.")
            break

        img = detector.findPose(img)
        lmList, bboxInfo = detector.findPosition(img)

        time += 1

        if bboxInfo:
            lm_right_hip = lmList[23]
            lm_left_hip = lmList[24]
            lm_right_knee = lmList[25]
            lm_left_knee = lmList[26]
            lm_left_ankle = lmList[28]
            lm_right_ankle = lmList[27]

            # Calculate distance between ankles
            point_2 = distance(lm_right_ankle, lm_left_ankle)

            # Apply exponential filter
            point_1_exp = 0.5 * point_1 + 0.5 * point_2
            #li.append(point_1_exp)

            if point_1_exp > threshold and point_1 < threshold:
                if time - peak_loc >= 12:  # 12 can be adjusted based on requirements
                    steps += 1
                    peak_loc = time
                    threshold = 0.25 * (maxi + mini) + 0.5 * threshold
                    print(threshold)
                    maxi = point_1
                    mini = point_1

            if point_1_exp > maxi:
                maxi = point_1_exp
            if point_1_exp < mini:
                mini = point_1_exp

            point_1 = point_1_exp
        elif not bboxInfo:
            print("Human not detected.")
            continue;

        # Display the image
        #cv2_imshow(img)

        # Break loop on ESC key press
        key = cv2.waitKey(1)
        if key == 27:
            break


    # Release resources
    cap.release()
    cv2.destroyAllWindows()
    return steps

In [13]:
cap = cv2.VideoCapture('/content/vid2.mp4')

detector = PoseDetector()

# Initialize threshold
try:
    threshold, point1 = initialize_threshold(cap, detector)
except ValueError as e:
    print(e)
else:
    # Main processing loop
    print(steps := main_processing_loop(cap, detector, threshold, point_1 = point1))


Human not detected.
Human not detected.
Human not detected.
Human not detected.


AttributeError: module 'google.protobuf.message_factory' has no attribute 'GetMessageClass'

In [ ]:
cap.release()
cv2.destroyAllWindows()
print(steps)
#plt.plot(li2+li)
# print(threshold)
#plt.show()

23
